Inspiration：https://mp.weixin.qq.com/s/f_JqsgTyqbKAZ-NQNf-wzA  
Reference：  
https://zhuanlan.zhihu.com/p/484812586  
https://blog.csdn.net/qq_44665162/article/details/121103280?spm=1001.2014.3001.5501

In [1]:
# for data scraping
#encoding:utf-8
import requests
from bs4 import BeautifulSoup
import time as ti
import csv
from lxml import etree
import re
from datetime import datetime
from googletrans import Translator
# for data analyzing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
#Part I: camouflage and request response:
def get_html(url):
    #Because many web pages have anti crawlers, we add hearders disguise
    #In the cat's eye movie web page -- F12 -- Network -- all -- 4 -- header -- find the user agent
    #Copy and paste the content
    headers = {  # 设置header
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'Cookie' : 'SUB=_2AkMU1fRvf8NxqwJRmP0QyG_qbIx_zwDEieKiiQW0JRMxHRl-yT9jqmshtRB6P1XagHnBRLbNMtnyKDtJL3UU5PnizHQ3; _s_tentry=-; Apache=5488155764905.276.1680638770371; SINAGLOBAL=5488155764905.276.1680638770371; ULV=1680638770378:1:1:1:5488155764905.276.1680638770371:; ab.storage.deviceId.a9882122-ac6c-486a-bc3b-fab39ef624c5=%7B%22g%22%3A%225c7f0b85-d8d6-8660-676e-d5c75b1f5fc9%22%2C%22c%22%3A1680638780350%2C%22l%22%3A1680638780350%7D'
    }
    #The purpose of responding to the request, combined with the disguise of hearders, is to let the server know that this is not a crawler, but a person
    #Get website information using get
    result = requests.get(url, headers = headers)
    #Because the crawler is fast, the server may make an error 403
    #Write a judgment, 200 is success
    if result.status_code == 200:
        #The response is successful and a string is returned
        return result.text
    return

In [3]:
def parsing_html(html):
    ti.sleep(1)
    #patter = re.compile('<dd>.*?board-index')
    bsSoup = BeautifulSoup(html, 'html.parser')
    #a = [x.find("i").text for x in bsSoup.find_all("dd")]
    items = bsSoup.find_all('tr')
    a = []
    
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')  # Get the current time
    
    # loop through each tr tag and extract the information
    domainName = 'https://s.weibo.com'
    for i in items[2:]:
        ti.sleep(0.1)
        rank = i.find('td', class_='td-01 ranktop').text
        a_tag = i.find('td', class_='td-02').find('a')
        href = domainName+a_tag['href']
        hotWord = a_tag.text
        hot_info = i.find('span').text
        parts = hot_info.split()
        if len(parts) > 1:
            type_info = parts[0]
            hotness = parts[1]
        else:
            type_info = '其他'
            hotness = hot_info
        ti.sleep(0.2)
        
        # translate chinese to english
        translator = Translator()
        hotWord_en = translator.translate(hotWord, dest='en').text
        type_info_en = translator.translate(type_info, dest='en').text

        c = {'Rank' : rank,
             'Time': now,  # Record the current time
             'Hot Word in Chinese': hotWord,
             'Hot Word in English' : hotWord_en,
             'Type Info in Chinese' : type_info,
             'Type Info in English' : type_info_en,
             'Hotness' : hotness,
             'Href' : href
            }
        a.append(c)
    return a

In [4]:
def write_to_file(content, headers=None):
    with open('weibo_hot_search.csv', 'a', encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        if not csvfile.tell():  # If the file is empty, write the column name
            writer.writeheader()
        writer.writerow(content)

In [5]:
def get_csv():
    url = 'https://s.weibo.com/top/summary'
    html = get_html(url)
    info_list = parsing_html(html)
    
    for item in parsing_html(html):
        headers = item.keys()
        write_to_file(item, headers=headers)

In [6]:
get_csv()

In [31]:
# Read data
df = pd.read_csv('weibo_hot_search.csv')

df.head()

,Rank,Time,Hot Word in Chinese,Hot Word in English,Type Info in Chinese,Type Info in English,Hotness,Href
0,1,2023-04-08 23:15:53,胃病转向胃癌有5个信号,Gastric disease has 5 signals to turn to gastr...,其他,other,1308773,https://s.weibo.com/weibo?q=%23%E8%83%83%E7%97...
1,2,2023-04-08 23:15:53,林俊杰买虚拟地产浮亏91%,Lin Junjie bought a virtual real estate floati...,其他,other,1000365,https://s.weibo.com/weibo?q=%23%E6%9E%97%E4%BF...
2,3,2023-04-08 23:15:53,这是现实版人在画中游,This is the real version of the person in the ...,其他,other,820829,https://s.weibo.com/weibo?q=%23%E8%BF%99%E6%98...
3,4,2023-04-08 23:15:53,被闹钟惊醒对身体危害大,Wake up by the alarm clock is harmful to the body,其他,other,662430,https://s.weibo.com/weibo?q=%23%E8%A2%AB%E9%97...
4,5,2023-04-08 23:15:53,罗云熙下海动图,Luo Yunxi goes down the sea,剧集,Drama,638454,https://s.weibo.com/weibo?q=%23%E7%BD%97%E4%BA...
